# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc_fonte = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc_fonte.shape)
sinasc_fonte.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2)

sinasc_fonte.isna().value_counts()


ORIGEM  CODESTAB  CODMUNNASC  LOCNASC  IDADEMAE  ESTCIVMAE  ESCMAE  CODOCUPMAE  QTDFILVIVO  QTDFILMORT  CODMUNRES  GESTACAO  GRAVIDEZ  PARTO  CONSULTAS  DTNASC  HORANASC  SEXO   APGAR1  APGAR5  RACACOR  PESO   IDANOMAL  DTCADASTRO  CODANOMAL  NUMEROLOTE  VERSAOSIST  DTRECEBIM  DIFDATA  DTRECORIGA  NATURALMAE  CODMUNNATU  CODUFNATU  ESCMAE2010  SERIESCMAE  DTNASCMAE  RACACORMAE  QTDGESTANT  QTDPARTNOR  QTDPARTCES  IDADEPAI  DTULTMENST  SEMAGESTAC  TPMETESTIM  CONSPRENAT  MESPRENAT  TPAPRESENT  STTRABPART  STCESPARTO  TPNASCASSI  TPFUNCRESP  TPDOCRESP  DTDECLARAC  ESCMAEAGR1  STDNEPIDEM  STDNNOVA  CODPAISRES  TPROBSON  PARIDADE  KOTELCHUCK  CONTADOR  munResStatus  munResTipo  munResNome  munResUf  munResLat  munResLon  munResAlt  munResArea
False   False     False       False    False     False      False   False       False       False       False      False     False     False  False      False   False     False  False   False   False    False  False     False       True       False   

In [3]:
# 3) 
colunas = ['LOCNASC','IDADEMAE','ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sisnasc = sinasc_fonte[colunas]



In [4]:
# 4)
sisnasc.isna().value_counts()


LOCNASC  IDADEMAE  ESTCIVMAE  ESCMAE  QTDFILVIVO  GESTACAO  GRAVIDEZ  CONSULTAS  APGAR5
False    False     False      False   False       False     False     False      False     23721
                                      True        False     False     False      False      1401
                                      False       True      False     False      False      1045
                   True       False   False       False     False     False      False       240
                   False      True    False       False     False     False      False       220
                              False   True        True      False     False      False       105
                                      False       False     False     False      True         74
                                                            True      False      False        49
                   True       True    False       False     False     False      False        30
                   False      True    F

In [5]:
# 5)
sisnasc = sisnasc[~sisnasc['APGAR5'].isna()]
sisnasc.value_counts()

LOCNASC  IDADEMAE  ESTCIVMAE  ESCMAE       QTDFILVIVO  GESTACAO           GRAVIDEZ  CONSULTAS  APGAR5
1        19        1.0        8 a 11 anos  0.0         37 a 41 semanas    Única     4          9.0       94
         18        1.0        8 a 11 anos  0.0         37 a 41 semanas    Única     4          9.0       92
         20        1.0        8 a 11 anos  0.0         37 a 41 semanas    Única     4          9.0       88
         17        1.0        8 a 11 anos  0.0         37 a 41 semanas    Única     4          9.0       84
         20        2.0        8 a 11 anos  0.0         37 a 41 semanas    Única     4          9.0       80
                                                                                                         ..
         26        5.0        4 a 7 anos   4.0         42 semanas e mais  Única     4          10.0       1
                                                       37 a 41 semanas    Única     3          10.0       1
                                  

In [6]:
# 6)

sisnasc = sisnasc.drop(sisnasc[sisnasc['ESTCIVMAE'] == 9].index)
sisnasc = sisnasc.drop(sisnasc[sisnasc['CONSULTAS'] == 9].index)

#sisnasc[sisnasc['ESTCIVMAE'] == 9].value_counts()
#sisnasc[sisnasc['CONSULTAS'] == 9].value_counts()
sisnasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [7]:
# 7)
sisnasc['QTDFILVIVO'].fillna(0, inplace=True)
sisnasc[sisnasc['QTDFILVIVO'].isna()]

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5


In [8]:
# 8)
sisnasc[sisnasc.isna()].value_counts()

Series([], dtype: int64)

In [9]:
# 9.1)

normal = 'asfixia_normal'
leve = 'asfixia_leve'
moderada = 'asfixia_moderada'
severa = 'asfixiaa_severa'

sisnasc.loc[(sisnasc['APGAR5'] >= 8),'NASCIMENTO'] = normal
sisnasc.loc[(sisnasc['APGAR5'] >= 6) & (sisnasc['APGAR5'] <= 7),'NASCIMENTO'] = leve
sisnasc.loc[(sisnasc['APGAR5'] >= 4) & (sisnasc['APGAR5'] <= 5),'NASCIMENTO'] = moderada
sisnasc.loc[(sisnasc['APGAR5'] >= 0) & (sisnasc['APGAR5'] <= 3),'NASCIMENTO'] = severa

sisnasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,NASCIMENTO
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,asfixia_normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,asfixia_normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,asfixia_normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,asfixia_normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,asfixia_normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,asfixia_normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,asfixia_normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,asfixia_normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,asfixia_normal


In [10]:
# 9.2)
sisnasc['NASCIMENTO'].value_counts()/sisnasc['NASCIMENTO'].shape[0]*100


asfixia_normal      98.282893
asfixia_leve         1.189630
asfixiaa_severa      0.273091
asfixia_moderada     0.254386
Name: NASCIMENTO, dtype: float64

In [13]:
# 9.2)

nasc_normal = sisnasc.loc[sisnasc['NASCIMENTO'] == normal]
nasc_leves = sisnasc.loc[sisnasc['NASCIMENTO'] == leve]
nasc_moderados = sisnasc.loc[sisnasc['NASCIMENTO'] == moderada]
nasc_severos = sisnasc.loc[sisnasc['NASCIMENTO'] == severa]

m_normal = round(nasc_normal.IDADEMAE.median())
m_leve = round(nasc_leves.IDADEMAE.median())
m_moderada = round(nasc_moderados.IDADEMAE.median())
m_severa = round(nasc_severos.IDADEMAE.median())

total = sisnasc.loc[sisnasc['ESTCIVMAE'] <= 4].shape[0]

s_normal = round((nasc_normal.loc[nasc_normal['ESTCIVMAE'] == 1].shape[0]/total * 100), 2)
c_normal = round((nasc_normal.loc[nasc_normal['ESTCIVMAE'] == 2].shape[0]/total * 100), 2)
v_normal = round((nasc_normal.loc[nasc_normal['ESTCIVMAE'] == 3].shape[0]/total * 100), 2)
d_normal = round((nasc_normal.loc[nasc_normal['ESTCIVMAE'] == 4].shape[0]/total * 100), 2)

s_leve = round((nasc_leves.loc[nasc_leves['ESTCIVMAE'] == 1].shape[0]/total * 100), 2)
c_leve = round((nasc_leves.loc[nasc_leves['ESTCIVMAE'] == 2].shape[0]/total * 100), 2)
v_leve = round((nasc_leves.loc[nasc_leves['ESTCIVMAE'] == 3].shape[0]/total * 100), 2)
d_leve = round((nasc_leves.loc[nasc_leves['ESTCIVMAE'] == 4].shape[0]/total * 100), 2)

s_moderado = round((nasc_moderados.loc[nasc_moderados['ESTCIVMAE'] == 1].shape[0]/total * 100), 2)
c_moderado = round((nasc_moderados.loc[nasc_moderados['ESTCIVMAE'] == 2].shape[0]/total * 100), 2)
v_moderado = round((nasc_moderados.loc[nasc_moderados['ESTCIVMAE'] == 3].shape[0]/total * 100), 2)
d_moderado = round((nasc_moderados.loc[nasc_moderados['ESTCIVMAE'] == 4].shape[0]/total * 100), 2)

s_severo = round((nasc_severos.loc[nasc_severos['ESTCIVMAE'] == 1].shape[0]/total * 100), 2)
c_severo = round((nasc_severos.loc[nasc_severos['ESTCIVMAE'] == 2].shape[0]/total * 100), 2)
v_severo = round((nasc_severos.loc[nasc_severos['ESTCIVMAE'] == 3].shape[0]/total * 100), 2)
d_severo = round((nasc_severos.loc[nasc_severos['ESTCIVMAE'] == 4].shape[0]/total * 100), 2)

In [14]:
medias = [m_normal, m_leve, m_moderada, m_severa]
solteiras = [s_normal, s_leve, s_moderado, s_severo]
casadas = [c_normal, c_leve, c_moderado, c_severo]
viuvas = [v_normal, v_leve, v_moderado, v_severo]
divociadas = [d_normal, d_leve, d_moderado, d_severo]

data = {
    "MAES_IDADE_MED": medias, 
    "MAES_SOLTEIRAS (%)": solteiras,
    "MAES_CASADAS (%)": casadas,
    "MAES_VIUVAS (%)": viuvas,
    "MAES_DIVORCIADAS (%)": divociadas
}

df = pd.DataFrame(
    data,
    index=[normal, leve, moderada,severa],

)
df

,MAES_IDADE_MED,MAES_SOLTEIRAS (%),MAES_CASADAS (%),MAES_VIUVAS (%),MAES_DIVORCIADAS (%)
asfixia_normal,26,45.26,50.50,0.3,2.30
asfixia_leve,25,0.57,0.58,0.0,0.03
asfixia_moderada,27,0.12,0.09,0.0,0.00
asfixiaa_severa,24,0.11,0.15,0.0,0.01
